In [175]:
import regrex as re


## **Descripicion del proyecto**

## Contexto de la base de datos

El éxito o la popularidad de un videojuego depende de una combinación de factores internos y externos. Entre ellos, el precio del juego influye en la decisión de compra, mientras que las calificaciones y reseñas, tanto positivas como negativas, afectan la percepción del público y su disposición a recomendarlo. El género del juego y los modos multijugador también juegan un papel importante, ya que determinan el tipo y tamaño de la audiencia, así como la actividad de la comunidad. Además, la fecha de lanzamiento puede ser determinante, evitando competencia o aprovechando temporadas clave, mientras que estrategias de marketing y promoción incrementan su visibilidad. Factores internos como la jugabilidad, historia, gráficos y música impactan indirectamente al influir en la satisfacción de los jugadores. Finalmente, tendencias culturales y la exposición en redes y plataformas de streaming pueden catapultar la popularidad de un título más allá de sus características técnicas, haciendo que el éxito sea el resultado de la interacción de múltiples variables a lo largo del tiempo.

## Descripicion general del contenido en la base de datos

se encuentran titulos desde el año 2013 hasta el año 2023, su fecha de lanzamiento, su precio normal en steam, su id en steam,su calificacion postiva y negativo, el minimo y el maximo de dueños de un juego y la duracion en horas de la campaña en solitario de un videojuego

## significado de cada columna

name(nombre del videojuego)

release_date(cuando salio el juego)

price(precio del videojuego)

positive(calificacion positiva del juego)

negative(calificacion negativa del juego)

app_id(numero que representa al juego)

min_owners(cantidad minima de dueños que tiene el juego)

max_owners(cantidad maxima de dueños que tuvo el juego)

hltb_single(la duracion en horas mde la campaña en solitario (es aproximado))

In [176]:
import pandas as pd
import numpy as np
import random

def ensuciar_df(
    df_original,
    porcentaje_duplicados=0.3,
    porcentaje_invalid=0.02,
    porcentaje_nan=0.03,
    max_nan_col=10,
    max_intentos=30,
    verbose=True
):
    """Ensucia un DataFrame introduciendo duplicados, NaNs, errores de tipo y valores inválidos."""
    df = df_original.copy()

    for intento in range(max_intentos):
        df_temp = df.copy()

        # ----- 1. Duplicar filas -----
        num_duplicados = random.randint(1, int(len(df_temp) * porcentaje_duplicados))
        duplicadas = df_temp.sample(n=num_duplicados, replace=True, random_state=random.randint(0, 9999))
        df_temp = pd.concat([df_temp, duplicadas], ignore_index=True)

        # ----- 2. Insertar valores inválidos -----
        num_invalid = int(len(df_temp) * porcentaje_invalid)
        columnas_invalid = random.sample(df_temp.columns.tolist(), max(1, len(df_temp.columns) // 2))
        for col in columnas_invalid:
            if df_temp[col].dtype != "object":
                df_temp[col] = df_temp[col].astype("object")

            valid_indices = df_temp.index[df_temp[col].notna()]
            if len(valid_indices) > 0:
                invalid_indices = np.random.choice(valid_indices, size=min(num_invalid, len(valid_indices)), replace=False)
                df_temp.loc[invalid_indices, col] = "Auto%#"

        # ----- 3. Cambiar tipos de columnas -----
        columnas_a_cambiar = random.sample(df_temp.columns.tolist(), max(1, len(df_temp.columns) // 2))
        for col in columnas_a_cambiar:
            if pd.api.types.is_numeric_dtype(df_temp[col]):
                df_temp[col] = df_temp[col].astype(str)
            else:
                df_temp[col] = pd.to_numeric(df_temp[col], errors="coerce")

        # ----- 4. Insertar NaNs -----
        num_nan = int(len(df_temp) * porcentaje_nan)
        for column in df_temp.columns:
            nan_indices = np.random.choice(df_temp.index, size=min(num_nan, len(df_temp)), replace=False)
            df_temp.loc[nan_indices, column] = np.nan

        # ----- 5. Validar límite de NaNs -----
        nan_porcentaje = df_temp.isnull().mean() * 100
        if (nan_porcentaje > max_nan_col).any():
            if verbose:
                print(f"Intento {intento + 1}: Columna con más del {max_nan_col}% de NaNs. Reintentando...")
            continue
        else:
            if verbose:
                print(f"✅ DataFrame ensuciado correctamente en el intento {intento + 1}.")
            return df_temp

    raise ValueError(f"No se logró generar un DataFrame válido en {max_intentos} intentos.")

#Cargar csv y lerlo
ruta_csv = 'https://raw.githubusercontent.com/ibrahimelCABALLOesquizofrenico/ProyectoLimpiezaDeBase/refs/heads/main/games_steam_2013_2023_es.csv'
df = pd.read_csv(ruta_csv)

df_sucio = ensuciar_df(df)

# Guardar el resultado
df_sucio.to_csv("df_sucio.csv", index=False)

# Reporte final
print("\n--- Reporte de NaNs ---")
print(df_sucio.isnull().sum())
print(f"\nTamaño final del DataFrame: {df_sucio.shape}")


Intento 1: Columna con más del 10% de NaNs. Reintentando...
✅ DataFrame ensuciado correctamente en el intento 2.

--- Reporte de NaNs ---
nombre                          2100
fecha_de_lanzamiento            2100
precio                          2100
reseñas_positivas               2100
reseñas_negativas               2100
id_del_juego                    3447
minimo_de_dueños                2100
maximo_de_dueños                3451
duracion_de_la_campaña_aprox    2100
dtype: int64

Tamaño final del DataFrame: (70005, 9)


In [177]:
import pandas as pd
import numpy as np
import random

def ensuciar_df(
    df_original,
    columnas_protegidas=None,
    porcentaje_duplicados=0.3,
    porcentaje_invalid=0.02,
    porcentaje_nan=0.05,
    valor_invalid='bbb'
):
    """Ensucia un DataFrame introduciendo duplicados, valores inválidos, cambios de tipo y NaNs,
    excluyendo las columnas protegidas."""
    
    df = df_original.copy()
    columnas_protegidas = columnas_protegidas or []
    columnas_modificables = [c for c in df.columns if c not in columnas_protegidas]

    # 1️⃣ Duplicar algunas filas aleatorias (esto afecta todas las columnas, pero no modifica su contenido)
    num_duplicados = random.randint(1, int(len(df) * porcentaje_duplicados))
    duplicadas = df.sample(n=num_duplicados, replace=True, random_state=random.randint(0, 9999))
    df = pd.concat([df, duplicadas], ignore_index=True)

    # 2️⃣ Insertar valores inválidos en columnas aleatorias (excepto las protegidas)
    num_invalid = int(len(df) * porcentaje_invalid)
    columnas_invalid = random.sample(columnas_modificables, max(1, len(columnas_modificables) // 2))
    for col in columnas_invalid:
        if df[col].dtype != "object":
            df[col] = df[col].astype("object")

        valid_indices = df.index[df[col].notna()]
        if len(valid_indices) > 0:
            invalid_indices = np.random.choice(valid_indices, size=min(num_invalid, len(valid_indices)), replace=False)
            df.loc[invalid_indices, col] = valor_invalid

    # 3️⃣ Cambiar tipos de formato en columnas aleatorias (excepto las protegidas)
    columnas_a_cambiar = random.sample(columnas_modificables, max(1, len(columnas_modificables) // 2))
    for col in columnas_a_cambiar:
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].astype(str)
        else:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # 4️⃣ Insertar NaNs en columnas modificables solamente
    num_nan = int(len(df) * porcentaje_nan)
    for column in columnas_modificables:
        nan_indices = np.random.choice(df.index, size=min(num_nan, len(df)), replace=False)
        df.loc[nan_indices, column] = np.nan

    return df

# Cargar el CSV y aplicar las modificaciones
ruta_csv = "https://raw.githubusercontent.com/ibrahimelCABALLOesquizofrenico/ProyectoLimpiezaDeBase/refs/heads/main/games_steam_2013_2023_es.csv"
df = pd.read_csv(ruta_csv)

# Columnas que no deben ensuciarse
columnas_protegidas = ["name", "release_date"]

# Intentos
max_intentos = 100

for intento in range(1, max_intentos + 1):
    df_sucio = ensuciar_df(df, columnas_protegidas=columnas_protegidas)

    # Calcular porcentaje de NaNs en columnas modificables
    nan_porcentaje = df_sucio.drop(columns=columnas_protegidas, errors="ignore").isnull().mean() * 100

    if (nan_porcentaje > 10).any():
        print(f"Intento {intento}: hay columnas con más del 10% de NaNs, reintentando...")
    else:
        print(f"✅ Finalización exitosa en el intento {intento}: no hay columnas con más del 10% de NaNs.")
        break
else:
    print("⚠️ Se alcanzó el número máximo de intentos sin lograr un DataFrame válido.")

# Guardar el DataFrame sucio
df_sucio.to_csv("df_sucio.csv", index=False)

# Mostrar resumen
print("\n--- Reporte final de NaNs ---")
print(df_sucio.isnull().sum())
print(f"\nTamaño final del DataFrame: {df_sucio.shape}")



Intento 1: hay columnas con más del 10% de NaNs, reintentando...
Intento 2: hay columnas con más del 10% de NaNs, reintentando...
Intento 3: hay columnas con más del 10% de NaNs, reintentando...
Intento 4: hay columnas con más del 10% de NaNs, reintentando...
✅ Finalización exitosa en el intento 5: no hay columnas con más del 10% de NaNs.

--- Reporte final de NaNs ---
nombre                          3950
fecha_de_lanzamiento            3950
precio                          5445
reseñas_positivas               3950
reseñas_negativas               3950
id_del_juego                    3950
minimo_de_dueños                5449
maximo_de_dueños                3950
duracion_de_la_campaña_aprox    3950
dtype: int64

Tamaño final del DataFrame: (79013, 9)


In [178]:
df_sucio.head()

,nombre,fecha_de_lanzamiento,precio,reseñas_positivas,reseñas_negativas,id_del_juego,minimo_de_dueños,maximo_de_dueños,duracion_de_la_campaña_aprox
0,bbb,"Oct 12, 2017",0.99,53.0,5.0,655370.0,0.0,20000,nan
1,Henosis™,bbb,5.99,3.0,0.0,1355720.0,0.0,20000,nan
2,Two Weeks in Painland,"Feb 3, 2020",0.00,50.0,8.0,1139950.0,0.0,20000,nan
3,Wartune Reborn,"Feb 26, 2021",0.00,87.0,49.0,1469160.0,50000.0,100000,nan
4,TD Worlds,"Jan 9, 2022",10.99,21.0,7.0,1659180.0,0.0,20000,nan


## **Inicio de la limpieza de la base de datos**

In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60952 entries, 0 to 60951
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   nombre                        60952 non-null  object 
 1   fecha_de_lanzamiento          60952 non-null  object 
 2   precio                        60952 non-null  float64
 3   reseñas_positivas             60952 non-null  int64  
 4   reseñas_negativas             60952 non-null  int64  
 5   id_del_juego                  60952 non-null  int64  
 6   minimo_de_dueños              60952 non-null  int64  
 7   maximo_de_dueños              60952 non-null  int64  
 8   duracion_de_la_campaña_aprox  12972 non-null  float64
dtypes: float64(2), int64(5), object(2)
memory usage: 4.2+ MB


In [180]:
df.shape

(60952, 9)

In [181]:
df_sucio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79013 entries, 0 to 79012
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   nombre                        75063 non-null  object 
 1   fecha_de_lanzamiento          75063 non-null  object 
 2   precio                        73568 non-null  float64
 3   reseñas_positivas             75063 non-null  float64
 4   reseñas_negativas             75063 non-null  float64
 5   id_del_juego                  75063 non-null  float64
 6   minimo_de_dueños              73564 non-null  float64
 7   maximo_de_dueños              75063 non-null  object 
 8   duracion_de_la_campaña_aprox  75063 non-null  object 
dtypes: float64(5), object(4)
memory usage: 5.4+ MB


In [182]:
df_sucio.isnull().sum()

nombre                          3950
fecha_de_lanzamiento            3950
precio                          5445
reseñas_positivas               3950
reseñas_negativas               3950
id_del_juego                    3950
minimo_de_dueños                5449
maximo_de_dueños                3950
duracion_de_la_campaña_aprox    3950
dtype: int64

In [183]:
df_sucio.shape

(79013, 9)

## **Codigo para remplazar las palabras basura,los nan y celdas vacias con ceros y eliminar filas y columnas repetidas**

In [184]:
#PD: Aqui si use IA profe, para que el codigo funcionara bien 
# --- Cargar el CSV sucio ---
df_limpio = pd.read_csv("df_sucio.csv")

# --- Reemplazar palabras basura ---
palabras_basura = ["bbb", "invalid", "Auto%#", "nan", "None", "NULL"]
df_limpio.replace(palabras_basura, 0, inplace=True)

# --- Reemplazar celdas vacías o espacios en blanco con 0 ---
df_limpio.replace(r'^\s*$', 0, regex=True, inplace=True)

# ---Reemplazar valores NaN con 0 ---
df_limpio.fillna(0, inplace=True)

# --- Eliminar filas y columnas completamente vacías ---
df_limpio.dropna(axis=0, how='all', inplace=True)  # Eliminar filas vacías
df_limpio.dropna(axis=1, how='all', inplace=True)  # Eliminar columnas vacías

# --- Eliminar columnas duplicadas ---
df_limpio = df.T.drop_duplicates().T

# --- Eliminar filas duplicadas ---
df_limpio.drop_duplicates(keep="first", inplace=True)

# --- Guardar el resultado limpio ---
df_limpio.to_csv("df_limpio.csv", index=False, encoding="utf-8-sig")

# --- Mostrar resumen ---
print("Limpieza completada exitosamente.")
print(f"Filas finales: {df_limpio.shape[0]}")
print(f"Columnas finales: {df_limpio.shape[1]}")
print("Archivo guardado como 'df_limpio.csv'")


Limpieza completada exitosamente.
Filas finales: 60952
Columnas finales: 9
Archivo guardado como 'df_limpio.csv'


## **Codigo para transformar de el tipo de dato de las columnas**

In [185]:
# --- Cargar el CSV limpio ---
df_limpio = pd.read_csv("df_limpio.csv")

# --- Columnas que queremos convertir a tipo entero ---
columnas_a_convertir = [
    "precio",
    "reseñas_positivas",
    "reseñas_negativas",
    "minimo_de_dueños",
    "maximo_de_dueños",
    "Duracion de la campaña(aprox)"
]

# --- Convertir esas columnas a entero ---
for col in columnas_a_convertir:
    if col in df_limpio.columns:
        # Primero convertir a numérico (por si hay texto o NaN)
        df_limpio[col] = pd.to_numeric(df_limpio[col], errors="coerce").fillna(0).astype(int)

# --- Guardar los cambios ---
df_limpio.to_csv("df_limpio.csv", index=False, encoding="utf-8-sig")

# --- Mostrar confirmación ---
print("Conversión a enteros completada correctamente.")
print("Archivo actualizado: 'df_limpio.csv'")


Conversión a enteros completada correctamente.
Archivo actualizado: 'df_limpio.csv'


**El codigo de arriba agrego nan a la columna de duración de la campaña por lo que se le aplicara otro fillna**

In [186]:
df_limpio.fillna(0, inplace=True)

## **Resultado final**

In [187]:
df_limpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60952 entries, 0 to 60951
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   nombre                        60952 non-null  object 
 1   fecha_de_lanzamiento          60952 non-null  object 
 2   precio                        60952 non-null  int64  
 3   reseñas_positivas             60952 non-null  int64  
 4   reseñas_negativas             60952 non-null  int64  
 5   id_del_juego                  60952 non-null  int64  
 6   minimo_de_dueños              60952 non-null  int64  
 7   maximo_de_dueños              60952 non-null  int64  
 8   duracion_de_la_campaña_aprox  60952 non-null  float64
dtypes: float64(1), int64(6), object(2)
memory usage: 4.2+ MB


In [188]:
df_limpio.shape

(60952, 9)

In [189]:
df_limpio.isnull().sum()

nombre                          0
fecha_de_lanzamiento            0
precio                          0
reseñas_positivas               0
reseñas_negativas               0
id_del_juego                    0
minimo_de_dueños                0
maximo_de_dueños                0
duracion_de_la_campaña_aprox    0
dtype: int64

In [190]:
df_limpio.duplicated().sum()

np.int64(0)

In [191]:
df_limpio.head()


,nombre,fecha_de_lanzamiento,precio,reseñas_positivas,reseñas_negativas,id_del_juego,minimo_de_dueños,maximo_de_dueños,duracion_de_la_campaña_aprox
0,Train Bandit,"Oct 12, 2017",0,53,5,655370,0,20000,0.0
1,Henosis™,"Jul 23, 2020",5,3,0,1355720,0,20000,0.0
2,Two Weeks in Painland,"Feb 3, 2020",0,50,8,1139950,0,20000,0.0
3,Wartune Reborn,"Feb 26, 2021",0,87,49,1469160,50000,100000,0.0
4,TD Worlds,"Jan 9, 2022",10,21,7,1659180,0,20000,0.0


## **Cargar csv limpio y sucio a mi carpeta local**

In [192]:
import random
# --- Función para ensuciar el DataFrame ---
def ensuciar_df(df_original, columnas_protegidas=None):
    df = df_original.copy()

    # Columnas que no deben ensuciarse
    if columnas_protegidas is None:
        columnas_protegidas = ['name', 'release_date']

    # --- Duplicar algunas filas ---
    num_duplicados = random.randint(1, int(len(df) * 0.3))
    duplicadas = df.sample(n=num_duplicados, replace=True)
    df = pd.concat([df, duplicadas], ignore_index=True)

    # --- Insertar valores 'bbb' en columnas no protegidas ---
    num_invalid = int(len(df) * 0.02)
    columnas_modificables = [c for c in df.columns if c not in columnas_protegidas]

    for col in random.sample(columnas_modificables, len(columnas_modificables) // 2):
        if df[col].dtype != 'object':
            df[col] = df[col].astype('object')
        valid_indices = df.index[df[col].notna()]
        if len(valid_indices) > 0:
            invalid_indices = np.random.choice(valid_indices, size=min(num_invalid, len(valid_indices)), replace=False)
            df.loc[invalid_indices, col] = 'bbb'

    # --- Cambiar tipos de formato en columnas no protegidas ---
    columnas_a_cambiar = random.sample(columnas_modificables, random.randint(1, len(columnas_modificables) // 2))
    for col in columnas_a_cambiar:
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].astype(str)
        else:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # --- Insertar NaNs en un 5% de las celdas, excepto las protegidas ---
    num_nan = int(len(df) * 0.05)
    for col in columnas_modificables:
        nan_indices = np.random.choice(df.index, size=min(num_nan, len(df)), replace=False)
        df.loc[nan_indices, col] = np.nan

    return df


# --- Cargar el CSV original ---
ruta_csv = 'https://raw.githubusercontent.com/ibrahimelCABALLOesquizofrenico/ProyectoLimpiezaDeBase/refs/heads/main/games_steam_2013_2023_es.csv'
df = pd.read_csv(ruta_csv)

# --- Aplicar el ensuciado ---
df_sucio = ensuciar_df(df, columnas_protegidas=['name', 'release_date'])

# --- Guardar el DataFrame sucio en un nuevo archivo CSV ---
df_sucio.to_csv('df_sucio.csv', index=False, encoding='utf-8-sig')

# --- Mensaje de confirmación ---
print("Base de datos sucia creada exitosamente como 'df_sucio.csv'")


Base de datos sucia creada exitosamente como 'df_sucio.csv'


In [193]:
import pandas as pd
import numpy as npa
# --- Función para limpiar el DataFrame ---
def limpiar_df(df):
    df_limpio = df.copy()

    # Reemplazar los valores "bbb" (o cualquier otro texto basura) por NaN
    df_limpio = df_limpio.replace('bbb', np.nan)

    # Reemplazar los NaN con 0
    df_limpio = df_limpio.fillna(0)

    # Asegurar que las columnas numéricas vuelvan a ser tipo numérico cuando sea posible
    for col in df_limpio.columns:
        df_limpio[col] = pd.to_numeric(df_limpio[col], errors='ignore')

    return df_limpio


# --- Cargar el CSV sucio ---
df_sucio = pd.read_csv('df_sucio.csv')

# --- Aplicar la limpieza ---
df_limpio = limpiar_df(df_sucio)

# --- Guardar el DataFrame limpio ---
df_limpio.to_csv('df_limpio.csv', index=False, encoding='utf-8-sig')

# --- Confirmación ---
print(" Base de datos limpia creada exitosamente como 'df_limpio.csv")


C:\Users\patri\AppData\Local\Temp\ipykernel_17736\3010420379.py:15: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_limpio[col] = pd.to_numeric(df_limpio[col], errors='ignore')


 Base de datos limpia creada exitosamente como 'df_limpio.csv


## **Traduccion de columnnas por medio de un diccionario y la funcion remplace**

# Traduccion o que significa cada columna en español

name -> nombre(nombre del videojuego)

release_date -> fecha de salida(cuando salio el juego)

price -> precio(precio del videojuego)

positive -> positivo(calificacion positiva del juego)

negative -> negativo(calificacion negativa del juego)

app_id -> id de la app(numero que representa al juego)

min_owners -> minimo de dueños(cantidad minima de dueños que tiene el juego)

max_owners -> maximo de dueños(cantidad maxima de dueños que tuvo el juego)

hltb_single -> duracion de canpaña en solitario(la duracion en horas mde la campaña en solitario (es aproximado))

In [194]:
# --- Diccionario de traducción ---
traducciones = {
    "name": "nombre",
    "release_date": "fecha_de_lanzamiento",
    "price": "precio",
    "positive": "calificacion_positivas",
    "negative": "calificacion_negativas",
    "app_id": "id_del_juego",
    "min_owners": "minimo_de_dueños",
    "max_owners": "maximo_de_dueños",
    "hltb_single": "duracion_de_la_campaña_aprox",
    }

# --- Función para traducir columnas ---
def traducir_columnas(df, diccionario):
    nuevos_nombres = {col: diccionario.get(col, col) for col in df.columns}
    df.rename(columns=nuevos_nombres, inplace=True)
    return df, nuevos_nombres

# --- Traducir el csv sucio ---
try:
    df_sucio = pd.read_csv("df_sucio.csv", encoding="utf-8")
    df_sucio, nombres_sucio = traducir_columnas(df_sucio, traducciones)
    df_sucio.to_csv("df_sucio_es.csv", index=False, encoding="utf-8-sig")
    print(" Archivo 'df_sucio.csv' traducido y guardado como 'df_sucio_es.csv'")
except FileNotFoundError:
    print("No se encontró el archivo 'df_sucio.csv'. Asegúrate de tenerlo en la misma carpeta.")

# --- Traducir el csv limpio ---
try:
    df_limpio = pd.read_csv("df_limpio.csv", encoding="utf-8")
    df_limpio, nombres_limpio = traducir_columnas(df_limpio, traducciones)
    df_limpio.to_csv("df_limpio_es.csv", index=False, encoding="utf-8-sig")
    print(" Archivo 'df_limpio.csv' traducido y guardado como 'df_limpio_es.csv'")
except FileNotFoundError:
    print(" No se encontró el archivo 'df_limpio.csv'. Asegúrate de tenerlo en la misma carpeta.")

# --- Mostrar resumen de traducciones ---
print("\nTraducciones aplicadas:")
for clave, valor in traducciones.items():
    print(f"{clave} → {valor}")


 Archivo 'df_sucio.csv' traducido y guardado como 'df_sucio_es.csv'
 Archivo 'df_limpio.csv' traducido y guardado como 'df_limpio_es.csv'

Traducciones aplicadas:
name → nombre
release_date → fecha_de_lanzamiento
price → precio
positive → calificacion_positivas
negative → calificacion_negativas
app_id → id_del_juego
min_owners → minimo_de_dueños
max_owners → maximo_de_dueños
hltb_single → duracion_de_la_campaña_aprox
